In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [17]:
class LinearRegression:
  def __init__(self, optimizer="none", regularization="none", learning_rate=None, epochs=None, batch_size=None, alpha=None, verbose=False):
    self.optimizer = optimizer
    self.regularization = regularization
    self.learning_rate = learning_rate
    self.epochs = epochs
    self.batch_size = batch_size
    self.alpha = alpha
    self.verbose = verbose
    self.weights = None
    self.bias = None

  def loss(self, y_pred, y_true):
        squared_diff = (y_pred - y_true) ** 2
        mse = squared_diff.mean()
        return mse

  def y_prediction(self, X, a, b):
        m, n = X.shape
        y_pred = X.dot(a) + b
        assert(y_pred.shape == (m,))
        return y_pred

  def gradient(self, X, a, b, y_true):
    m, n = X.shape
    y_pred = self.y_prediction(X, a, b)
    da = (2 / m) * np.dot(X.T, y_pred - y_true)
    db = (2 / m) * np.sum(y_pred - y_true)
    return da,db

  def vanilla_gradient_descent(self, X, y_true):
    m, n = X.shape
    loss_mse = []
    a = np.random.normal(0, 0.5, size=(n,)) * np.sqrt(2 / n)
    b = 0
    for i in range(self.epochs):
      da, db = self.gradient(X, a, b, y_true)
      a -= self.learning_rate * da  # Update weights a
      b -= self.learning_rate * db  # Update bias b
      y_pred = self.y_prediction(X, a, b)
      l_mse = self.loss(y_pred, y_true)
      loss_mse.append(l_mse)

      if self.verbose:
        print(f"Epoch {i+1}: Loss = {l_mse}")

  def mini_batch_gradient_descent(self, X, y_true):
    m, n = X.shape
    loss_mse = []
    a = np.random.normal(0, 0.5, size=(n,)) * np.sqrt(2 / n)
    b = 0

    for i in range(self.epochs):
      # Shuffle the data and create minibatches
      indices = np.random.permutation(m)
      X_shuffled = X[indices]
      y_shuffled = y_true[indices]

      for j in range(0, m, self.batch_size):
        X_batch = X_shuffled[j:j + self.batch_size]
        y_batch = y_shuffled[j:j + self.batch_size]

        da, db = self.gradient(X_batch, a, b, y_batch)
        a -= self.learning_rate * da  # Update weights a
        b -= self.learning_rate * db  # Update bias b

      y_pred = self.y_prediction(X, a, b)
      l_mse = self.loss(y_pred, y_true)
      loss_mse.append(l_mse)

      if self.verbose:
        print(f"Epoch {i+1}: Loss = {l_mse}")

  def stochastic_gradient_descent(self, X, y_true):
    m, n = X.shape
    loss_mse = []
    a = np.random.normal(0, 0.5, size=(n,)) * np.sqrt(2 / n)
    b = 0

    for i in range(self.epochs):
      for j in range(m):
        # Select a random data point
        random_index = np.random.randint(0, m)
        X_single = X[random_index:random_index + 1]
        y_single = y_true[random_index:random_index + 1]

        da, db = self.gradient(X_single, a, b, y_single)
        a -= self.learning_rate * da  # Update weights a
        b -= self.learning_rate * db  # Update bias b

      y_pred = self.y_prediction(X, a, b)
      l_mse = self.loss(y_pred, y_true)
      loss_mse.append(l_mse)

      if self.verbose:
        print(f"Epoch {i+1}: Loss = {l_mse}")

  # For Ridge Loss (L2 Regularization)
  def ridge_loss(self, X, y_true, a, b):
    # Compute predictions
    y_pred = self.y_prediction(X, a, b)

    # Compute the Mean Squared Error (MSE) loss
    mse_loss = np.mean((y_pred - y_true) ** 2)

    # Add Ridge regularization (L2) term
    reg_loss = self.alpha * np.sum(a ** 2)  # Ridge (L2) penalty
    return mse_loss + reg_loss

  # For Lasso Loss (L1 Regularization)
  def lasso_loss(self, X, y_true, a, b):
    # Compute predictions
    y_pred = self.y_prediction(X, a, b)

    # Compute the Mean Squared Error (MSE) loss
    mse_loss = np.mean((y_pred - y_true) ** 2)

    # Add Lasso regularization (L1) term
    reg_loss = self.alpha * np.sum(np.abs(a))  # Lasso (L1) penalty
    return mse_loss + reg_loss

  # For Ridge Gradient (L2 Regularization)
  def ridge_gradient(self, X, a, b, y_true):
    m, n = X.shape
    y_pred = self.y_prediction(X, a, b)

    # Compute the gradient of the loss function (MSE)
    da = (2 / m) * np.dot(X.T, (y_pred - y_true))
    db = (2 / m) * np.sum(y_pred - y_true)

    # Add Ridge (L2) regularization term to the gradient
    da += 2 * self.alpha * a  # Ridge regularization gradient term

    return da, db

  # For Lasso Gradient (L1 Regularization)
  def lasso_gradient(self, X, a, b, y_true):
    m, n = X.shape
    y_pred = self.y_prediction(X, a, b)

    # Compute the gradient of the loss function (MSE)
    da = (2 / m) * np.dot(X.T, (y_pred - y_true))
    db = (2 / m) * np.sum(y_pred - y_true)

    # Add Lasso (L1) regularization term to the gradient
    da += self.alpha * np.sign(a)  # Lasso regularization gradient term

    return da, db

  def fit(self, X, y):
    m, n = X.shape
    # Initialize weights and bias
    a = np.random.normal(0, 0.5, size=(n,)) * np.sqrt(2 / n)
    b = 0

    for epoch in range(self.epochs):
        y_pred = self.y_prediction(X, a, b)
        # Choose the loss and gradient function based on the regularization type
        if self.regularization == 'ridge':
            loss_value = self.ridge_loss(X, y, a, b)
            da, db = self.ridge_gradient(X, a, b, y)
        elif self.regularization == 'lasso':
            loss_value = self.lasso_loss(X, y, a, b)
            da, db = self.lasso_gradient(X, a, b, y)
        else:
            # No regularization (vanilla gradient descent)
            loss_value = self.loss(y_pred,y)  # Original MSE loss without regularization
            da, db = self.gradient(X, a, b, y)

        # Update parameters
        a -= self.learning_rate * da
        b -= self.learning_rate * db

        if self.verbose:
            print(f"Epoch {epoch+1}: Loss = {loss_value}")

In [30]:
def get_user_input():
    # Get user input for gradient descent type
    optimizer = input("Choose gradient descent method (vanilla, mini_batch, stochastic): ").strip().lower()
    if optimizer not in ['vanilla', 'mini_batch', 'stochastic']:
        raise ValueError("Invalid optimizer choice! Choose 'vanilla', 'mini_batch', or 'stochastic'.")

    # Get user input for regularization type
    regularization = input("Choose regularization method (none, ridge, lasso): ").strip().lower()
    if regularization not in ['none', 'ridge', 'lasso']:
        raise ValueError("Invalid regularization choice! Choose 'none', 'ridge', or 'lasso'.")

    # Get user input for alpha (regularization strength)
    alpha = None
    if regularization in ['ridge', 'lasso']:
        alpha = float(input("Enter regularization strength (alpha): ").strip())

    # Get user input for learning rate and number of epochs
    learning_rate = float(input("Enter learning rate: ").strip())
    epochs = int(input("Enter number of epochs: ").strip())

    # Get user input for batch size (for mini-batch and stochastic gradient descent)
    batch_size = None
    if optimizer == 'mini_batch':
        batch_size = int(input("Enter batch size for mini-batch gradient descent: ").strip())

    return optimizer, regularization, alpha, learning_rate, epochs, batch_size


def run_model_on_input(X_train, y_train):
    # Get the user inputs
    optimizer, regularization, alpha, learning_rate, epochs, batch_size = get_user_input()

    # Initialize the model with the selected options
    model = LinearRegression(
        optimizer=optimizer,
        regularization=regularization,
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size,
        alpha=alpha,
        verbose=True  # Set to True if you want verbose output for each epoch
    )

    # Fit the model on the dataset
    model.fit(X_train, y_train)


In [15]:
from google.colab import files
uploaded = files.upload()

Saving Admission_Predict.csv to Admission_Predict (1).csv


In [31]:
df=pd.read_csv("Admission_Predict.csv")
data_df=df.copy()
arr=np.array(data_df.drop('Chance of Admit ',axis=1))
arr=np.array(data_df.drop('Serial No.',axis=1))

mean_std_arr = np.vstack([np.mean(arr, axis=1), np.std(arr, axis=1)]).T

for i in range(len(arr)):
    arr[i] = (arr[i]-mean_std_arr[i,0])/mean_std_arr[i,1]

y_train = np.array(data_df['Chance of Admit '])  # extract the price column from data
X_train = arr

run_model_on_input(X_train,y_train)

Choose gradient descent method (vanilla, mini_batch, stochastic): vanilla
Choose regularization method (none, ridge, lasso): none
Enter learning rate: 0.0001
Enter number of epochs: 10
Epoch 1: Loss = 0.025147108083914987
Epoch 2: Loss = 0.025129351955612503
Epoch 3: Loss = 0.025111659679596565
Epoch 4: Loss = 0.025094031026231853
Epoch 5: Loss = 0.0250764657667089
Epoch 6: Loss = 0.02505896367304107
Epoch 7: Loss = 0.025041524518061707
Epoch 8: Loss = 0.02502414807542106
Epoch 9: Loss = 0.025006834119583498
Epoch 10: Loss = 0.02498958242582439
